# SSH Eigenmodes

In this example we consider SSH model which Hamiltonian can be express in the following way,

$$H=-\sum_{ i} v (A_{i}^{\dagger}B_{i}+h.c.)-\sum_{j}w(A^{\dagger}_{i}A_{i+1}+B^{\dagger}_{i}B_{i+1}+h.c. ),$$
here $v$ ($w$) is the overlap integral between the atoms in unit cell (betwens cells).  

In [112]:
!pip install kwant

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
import kwant
import matplotlib.pyplot as plt
import scipy.sparse.linalg as sla
import scipy as scp
import numpy as np
import scipy
from kwant.physics import dispersion
from numpy.linalg import eig

First of all we have to define our system. 

In [114]:
L = 50
# Building SSH model
def ssh_model(v=1, w=1, L=L, return_only_ham=1):

    syst = kwant.Builder()
    a = 1
    lat = kwant.lattice.chain(a)

    # Define the scattering region
    for n in range(L):
        syst[lat(n)] = 0

    # Left hopping
    for n in range(L):
        if n % 2:
            syst[lat(n - 1), lat(n)] = v

    # Right hopping
    for n in range(1, L):
        if not n % 2:
            syst[lat(n - 1), lat(n)] = w

    syst = syst.finalized()

    if return_only_ham:
        return syst.hamiltonian_submatrix(sparse=True)
    else:
        return syst.hamiltonian_submatrix(sparse=False) 





In our case we put onsite equal to zero. In order to defne our system we establish diferent hoppings for odd and even sites. For even sites the Hamiltonian part discrebes intercells hoppings is
$$-\sum_{ i} v (A_{i}^{\dagger}B_{i}+h.c.)$$
while for odds we have got
$$-\sum_{j}w(A^{\dagger}_{i}A_{i+1}+B^{\dagger}_{i}B_{i+1}+h.c. )$$

The SSH is the simplest one dimensional model in which one can observed edge states. We can observed this on the folowing plot,


In [115]:
from   ipywidgets import *

def SSH_energy(v,w):
  w1,v1=eig(ssh_model(w=w, v=v, L=L, return_only_ham=0))
  plt.plot(np.arange(0,L,1),sorted(np.real(w1)),'.')


interact(SSH_energy,v=(0,1,0.1),w=(0,1,0.1));

interactive(children=(FloatSlider(value=0.0, description='v', max=1.0), FloatSlider(value=0.0, description='w'…

One can observed that for $w>v$ there are two zero energy states which coresponds to edge states. The existance of such a states is topological protected. 

Now we want to plot energy spectrum as a function os hopping parameter $v$ when w is held constant. We have to identify each energy corespond to a given site. To this end, we used the following procedure. For a given $v$ we find all eigenvalues and eigenstates for every sites. Then we moved to another value of $v$ and do this same. Now to identife which eigenvalues corespond to this same site we chack for every eigenvector from the previous iteration gives us the bigest overlap (the bigest value of scalar product) with one of the state. The prosedure countinous for every value $v$.

In [116]:
def find_continuator(w: list, next_vectors: list) -> int:
    max_product = np.dot(w, next_vectors[0])
    max_index = 0

    for i in range(len(next_vectors)):
        product = np.abs(np.dot(w, next_vectors[i]))
        if product > max_product:
            max_product = product
            max_index = i
    return max_index


def plot_probability(k_tab: list ,eig_number: int,w,  show_unsorted=False) -> None:
    e_val_matrix = np.empty(shape=(len(k_tab), eig_number))
    e_vec_matrix = np.empty(shape=(len(k_tab), eig_number, L), dtype=complex)

    for i in range(len(k_tab)):
        v = k_tab[i]
        ham = ssh_model(v, w)
        e_val, e_vec = scipy.sparse.linalg.eigsh(
            ham, k=eig_number, sigma=0.01, which="LM", return_eigenvectors=True
        )

        e_val_matrix[i] = e_val
        for j in range(eig_number):
            e_vec_matrix[i, j] = e_vec[:, j]
            e_vec_matrix[i, j] = e_vec_matrix[i, j] / np.sqrt(
                np.dot(e_vec_matrix[i, j], e_vec_matrix[i, j])
            )

    for j in range(eig_number):
        pasek = [e_val_matrix[0, j]]
        wektory = [e_vec_matrix[0, j]]

        for i in range(0, len(k_tab) - 1):
            index = find_continuator(wektory[i], e_vec_matrix[i + 1])
            wektory.append(e_vec_matrix[i + 1, index])
            pasek.append(e_val_matrix[i + 1, index])

        plt.plot(k_tab, pasek, label=j)
        if show_unsorted:
            plt.scatter(k_tab, e_val_matrix[:, j], marker=".")

def SSH_spectrum_manipulate(w):
  plot_probability(k_tab=k,w=w, eig_number=energies_number)

k = np.linspace(0, 0.75, 50)  # Range of k we want to plot
energies_number = 47  # Number of energy bands to show

interact(SSH_spectrum_manipulate,w=(0.1,0.5,0.1));
plt.savefig("SSH_spectrum_sort.pdf")
plt.show()


interactive(children=(FloatSlider(value=0.30000000000000004, description='w', max=0.5, min=0.1), Output()), _d…

<Figure size 432x288 with 0 Axes>

We can easly diagonalize our Hamiltonian by teaking Fourier transform,
$$A^{\dagger}_{i}=\frac{1}{\sqrt{N}}\sum_{k}e^{ik\cdot x_{i}} A^{\dagger}_{k} \hspace{0.5cm} \mathrm{etc..}$$
In the end Hamiltonian in momentum space takes form
$$H=\sum_{k}(A^{\dagger}_{k},B^{\dagger}_{k})\begin{pmatrix}
0 & v+e^{ik}w \\
v+e^{-ik}w & 0 
\end{pmatrix}
\begin{pmatrix}
A_{k}  \\
B_{k} 
\end{pmatrix} $$

Now it is easy to find dispersion relation which in this case is given by the following expresion,
$$E(k)=\pm \sqrt{v^2+w^2+2vw\cos{k}} .$$

Now because we have used Fourier transform we tacitly impose boundary conditions. To procedure further we have to bild an infinit system. In order to do this, we need to add leads. 

In [117]:
def ssh_model_with_leads(t_1=1.0, t_2=0.5):
    L=200
    a=1
    # Start with an empty tight-binding system and a single square lattice.
    # `a` is the lattice constant (by default set to 1 for simplicity.
    syst = kwant.Builder()
    
    lat = kwant.lattice.Polyatomic([[2*a, 0]], [[0, 0], [a, 0]])
    lat.a, lat.b = lat.sublattices
    
    for n in range(L):
        syst[lat.a(n)] = 0
        syst[lat.b(n)] = 0
    
    # Left hopping
    for n in range(L):
        syst[lat.a(n), lat.b(n)] = t_1
        
    # Left hopping
    for n in range(1,L):
        syst[lat.b(n-1), lat.a(n)] = t_2
        
    leadless=syst
    leadless=leadless.finalized()
    leadless=leadless.hamiltonian_submatrix(sparse=True)


    sym_left_lead = kwant.TranslationalSymmetry([-2*a, 0])
    left_lead = kwant.Builder(sym_left_lead)
    left_lead[lat.a(0)] = 0
    left_lead[lat.b(0)] = 0
    left_lead[lat.a(0), lat.b(0)] = t_1
    left_lead[lat.b(0), lat.a(-1)] = t_2
    syst.attach_lead(left_lead)
    left_lead_fin = left_lead.finalized()

    sym_right_lead = kwant.TranslationalSymmetry([2*a, 0])
    right_lead = kwant.Builder(sym_right_lead)
    right_lead[lat.a(0)] = 0
    right_lead[lat.b(0)] = 0
    right_lead[lat.a(0), lat.b(0)] = t_1
    right_lead[lat.a(0), lat.a(1)] = t_2
    syst.attach_lead(right_lead)
    right_lead_fin = right_lead.finalized()
           
    syst = syst.finalized()
    return syst, left_lead_fin, right_lead_fin,leadless

syst, left_lead, right_lead, leadless= ssh_model_with_leads()

In [118]:
def plot_bandstructure(v,w):
    
    momenta=np.linspace(-np.pi, np.pi, 50)
    syst, left_lead, right_lead, leadless=ssh_model_with_leads(t_1=v, t_2=w)
    bands = kwant.physics.Bands(left_lead)
    energies = [bands(k) for k in momenta]
    
    plt.figure()
    plt.plot(momenta, energies)
    plt.xlabel("momentum [(lattice constant)^-1]")
    plt.ylabel("energy [t]")
    plt.savefig('ssh_energy_bands.pdf')
    plt.show()




In [119]:
def SSH_spectrum_manipulate_momentu_space(v,w):
  plot_probability(k_tab=k,w=w, eig_number=energies_number)

k = np.linspace(0, 0.75, 50)  # Range of k we want to plot
energies_number = 47  # Number of energy bands to show

interact(plot_bandstructure,v=(0.0,0.5,0.01),w=(0.0,0.5,0.01));
plt.show()


interactive(children=(FloatSlider(value=0.25, description='v', max=0.5, step=0.01), FloatSlider(value=0.25, de…

In [120]:
def ssh_model(v = 1, w = 1, L = 100, return_only_ham = 1):
    
    syst = kwant.Builder()
    a = 1
    lat = kwant.lattice.chain(a)

    # Define the scattering region
    for n in range(L):
        syst[lat(n)] = 0

    # Left hopping
    for n in range(L):
        if n%2:
            syst[lat(n-1), lat(n)] = v

    # Right hopping
    for n in range(1,L):      
        if not n%2:
            syst[lat(n-1), lat(n)] = w

    sym_left_lead = kwant.TranslationalSymmetry([-a])
    left_lead = kwant.Builder(sym_left_lead)
    left_lead[lat(0)] = 0
    left_lead[lat(1), lat(0)] = v
    left_lead[lat(2), lat(1)] = w
    syst.attach_lead(left_lead)
    left_lead_fin = left_lead.finalized()

    sym_right_lead = kwant.TranslationalSymmetry([a])
    right_lead = kwant.Builder(sym_right_lead)
    right_lead[lat(0)] = 0
    right_lead[lat(1), lat(0)] = v
    right_lead[lat(2), lat(1)] = w
    syst.attach_lead(right_lead)
    right_lead_fin = right_lead.finalized()

    #kwant.plot(syst)
    syst = syst.finalized()
    
    if(return_only_ham):
        return syst.hamiltonian_submatrix(sparse=False)
    else:
        return syst, syst.hamiltonian_submatrix(sparse=True)
    
# Plots energy spectrum
kwant.plotter.spectrum(ssh_model, x=('v', np.linspace(0,4,50)), file = 'energy_spectrum_ssh.png', dpi = 300)

# Plots wavefunction of the system for given energy
syst, ham = ssh_model(0.5, 1, return_only_ham = 0)

In [130]:
def plot_probability(v,w,energy):
    syst, ham = ssh_model(v=v, w=w, return_only_ham = 0)
    wf = kwant.solvers.default.wave_function(syst, energy)
    L=100
    p_1 = []
    p_2 = []
    for i in range(len(wf(0)[0])):
        p_1.append(abs(wf(0)[0][i])**2)
        p_2.append(abs(wf(1)[0][i])**2)
    plt.plot(np.linspace(0,L,L), p_1, '-.')
    plt.plot(np.linspace(0,L,L), p_2,  '-.')



In [131]:

interact(plot_probability,v=(0,0.9,0.01),w=(0,0.9,0.01),energy=(0,1,0.1))

interactive(children=(FloatSlider(value=0.45, description='v', max=0.9, step=0.01), FloatSlider(value=0.45, de…

<function __main__.plot_probability(v, w, energy)>